# Analyzing the improved genome assembly

## run the notebook serve on a client node

```
    srun --pty --nodes=1  --ntasks-per-node=1 --cpus-per-task=28 --time 100:00:00 --job-name bash-jupyter bash
    conda activate ont_assembly
    jupyter notebook --ip 0.0.0.0 --port 3001 --no-browser
```

## bokeh imports

In [1]:
from bokeh.plotting import figure, output_file, save
from bokeh.palettes import viridis
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, FactorRange, LabelSet, Whisker
from bokeh.models.tickers import FixedTicker
from bokeh.transform import jitter
from bokeh.layouts import column, row, gridplot
from math import pi
from bokeh.layouts import row
import random
import re

COLOR_PALETTE = ["#0072B2", "#D55E00", "#009E73", "#E69F00", "#CC79A7", "#56B4E9", "#F0E442"]

output_notebook()

Loading BokehJS ...

## load the data

In [5]:
def load_dist_dev(file_name, max_dist=None):
    names = []
    ret = []
    with open(file_name, "r") as in_file:
        for line in in_file:
            if line[0] == "#":
                continue
            else:
                # read_name distance_deviation expected_distance chr rpos1 rpos2 strand
                readname, distance, expected, chr, pos1, pos2, strnd = line.strip().split()
                if max_dist is None or int(distance) < max_dist:
                    names.append(readname)
                    chr = chr[:-len("_Tb427v10")]
                    ret.append([int(distance), int(expected), chr, int(pos1), int(pos2), strnd])
    return names, ret

ref_names, ref_dev = load_dist_dev("../data/out/virtual_paired_read_dist/referece.distance_deviation")


gap_pos = {}

with open("../data/in/genome_in/HGAP3_Tb427v10_diploid/HGAP3_Tb427v10_diploid_scaffolded.fasta.gaps.gff3", "r") as in_file:
    for line in in_file:
        if line[0] == "#":
            continue
        contig, source, anno_type, start, end, *extra = line.strip().split()
        contig = contig[:-len("_Tb427v10")]
        
        gap_name = contig + ":" + str(int(start)//1000) + "kbp"
        gap_pos[gap_name] = [contig, int(start), int(end)]


# identify collapsed regions 

# post process the data

In [3]:
f = figure(x_axis_label="genome position", y_axis_label="distance deviation")

xs = []
ys = []
r_names = []
data = []
for r_name, (distance, expected, chr, pos1, pos2, strnd) in zip(ref_names, ref_dev):
    if distance < -100:
        if "Chr5_A" in chr: # :1507kbp
            xs.append([min(pos1, pos2), max(pos1, pos2)])
            ys.append([distance, distance])
            r_names.append(r_name)
        data.append([chr, min(pos1, pos2), max(pos1, pos2), distance, r_name])

f.multi_line(xs=xs, ys=ys)

show(f, notebook_handle=True)

In [6]:
def percentile(l, p=0.05):
    l.sort()
    return l[int(len(l) * p)]

def cluster(data, distance_y=500, min_reads=5, max_cluster_size=50000):
    data.sort(key=lambda x: (x[0], x[1], x[3]))
    clusters = []
    for chr, start, end, deviation, r_name in data:
        fits_in_cluster = False
        for cluster in clusters:
            for cluster_chr, cluster_start, cluster_end, cluster_deviation, _ in cluster:
                if cluster_chr == chr and end >= cluster_start and start <= cluster_end and abs(deviation - cluster_deviation) <= distance_y:
                    fits_in_cluster = True
                    break
            if fits_in_cluster:
                cluster.append([chr, start, end, deviation, r_name])
                break
        if not fits_in_cluster:
            clusters.append([])
        clusters[-1].append([chr, start, end, deviation, r_name])

    processed_clusters = []
    for cluster in clusters:
        if len(cluster) > min_reads and not "unitig" in cluster[0][0]:
            cluster_start = percentile([x[1] for x in cluster], 0.05)
            cluster_end = percentile([x[1] for x in cluster], 0.95)
            if abs(cluster_start - cluster_end) > max_cluster_size: # filter out extremely large clusters
                continue
            assert cluster_start < cluster_end
            cluster_deviation = sum([x[3] for x in cluster]) / len(cluster)
            #print("cluster from", cluster_start, "to", cluster_end, "with deviation", cluster_deviation, "and", 
            #    len(cluster), "reads")
            processed_clusters.append([cluster[0][0], cluster_start, cluster_end, cluster_deviation, cluster])

    return processed_clusters


clusters = cluster(data)



cluster_overlapping_gap = []
new_cluster = []
for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in clusters:
    found_gap = False
    for gap_name, (gap_chr, gap_start, gap_end) in gap_pos.items():
        if cluster_chr == gap_chr and cluster_start <= gap_end and cluster_end >= gap_start:
            cluster_overlapping_gap.append([cluster_chr, cluster_start, cluster_end, cluster_deviation, c])
            found_gap = True
            break
    if not found_gap:
        new_cluster.append([cluster_chr, cluster_start, cluster_end, cluster_deviation, c])

print("clusters overlapping gaps")
for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in cluster_overlapping_gap:
    print(cluster_chr, int(cluster_start/1000), "-", int(cluster_end/1000), "k")
print()

print("new clusters")
for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in new_cluster:
    print(cluster_chr, int(cluster_start/1000), "-", int(cluster_end/1000), "k")
print()

def has_overlapping_clusters(clusters):
    for a, b in zip(clusters[:-1], clusters[1:]):
        if a[0] == b[0] and a[2] >= b[1]:
            return True
    return False

def merge_overlapping_clusters(new_cluster):
    ret = []
    for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in new_cluster:
        if len(ret) == 0 or ret[-1][0] != cluster_chr or ret[-1][2] < cluster_start:
            ret.append([cluster_chr, cluster_start, cluster_end, cluster_deviation, c])
        else:
            ret[-1][2] = max(ret[-1][2], cluster_end)
            ret[-1][4] = ret[-1][4] + c
            ret[-1][3] = sum([x[3] for x in ret[-1][4]]) / len(ret[-1][4])
    return ret

new_cluster.sort(key=lambda x: (x[0], x[1]))

while has_overlapping_clusters(new_cluster):
    new_cluster = merge_overlapping_clusters(new_cluster)

with open("../data/in/analysis_in/mis_assemblies.gff", "w") as file_out:
    for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in new_cluster:
        file_out.write("\t".join([cluster_chr + "_Tb427v10", ".", "misassembly", str(cluster_start), str(cluster_end), 
                                  ".", ".", ".", ""]) + "\n" )

gap_without_cluster = {}
for gap_name, (gap_chr, gap_start, gap_end) in gap_pos.items():
    found_gap = False
    for cluster_chr, cluster_start, cluster_end, cluster_deviation, c in cluster_overlapping_gap:
        if cluster_chr == gap_chr and cluster_start <= gap_end and cluster_end >= gap_start:
            found_gap = True
            break
    if not found_gap:
        gap_without_cluster[gap_name] = [gap_chr, gap_start, gap_end]

print("gaps without cluster")
for g in gap_without_cluster.keys():
    print(g)
print()

clusters overlapping gaps
Chr11_B 4913 - 4959 k
Chr11_B 4937 - 4957 k
Chr11_B 4946 - 4958 k
Chr1_A 943 - 987 k
Chr5_A 1662 - 1676 k

new clusters
BES11 42 - 49 k
BES14 11 - 58 k
BES17 24 - 61 k
Chr10_A 25 - 64 k
Chr10_A 5066 - 5068 k
Chr10_B 17 - 58 k
Chr10_B 36 - 58 k
Chr10_B 3101 - 3126 k
Chr10_B 5272 - 5303 k
Chr10_B 5288 - 5335 k
Chr11_A 0 - 10 k
Chr11_A 20 - 25 k
Chr11_A 574 - 615 k
Chr11_A 739 - 783 k
Chr11_A 2077 - 2101 k
Chr11_A 2130 - 2149 k
Chr11_A 2144 - 2153 k
Chr11_A 4935 - 4969 k
Chr11_A 4954 - 4968 k
Chr11_B 4646 - 4665 k
Chr11_B 4662 - 4665 k
Chr11_B 4725 - 4751 k
Chr11_B 4957 - 4958 k
Chr11_B 4958 - 4959 k
Chr11_B 4979 - 5016 k
Chr1_A 948 - 986 k
Chr1_A 2856 - 2897 k
Chr1_B 1270 - 1289 k
Chr1_B 1399 - 1441 k
Chr2_A 585 - 627 k
Chr2_B 370 - 404 k
Chr3_A 462 - 476 k
Chr3_A 476 - 495 k
Chr3_A 504 - 519 k
Chr3_A 508 - 540 k
Chr3_B 305 - 333 k
Chr5_A 771 - 813 k
Chr5_A 791 - 813 k
Chr5_A 879 - 902 k
Chr5_A 1510 - 1542 k
Chr5_A 1647 - 1669 k
Chr5_A 1653 - 1661 k
Chr5_A 1778 

# analyze the fixing of collapsed regions

## load the remaining data

In [109]:
fixed_names, fixed_dev = load_dist_dev("../data/out/virtual_paired_read_dist/fixed_n.distance_deviation")

gap_spanning_reads = {}

with open("../data/out/virtual_paired_read_dist/gap_spanning_reads", "r") as file_in:
    for line in file_in:
        readname, *gaps = line[:-1].strip().split()
        gap_names = []
        for gap in gaps:
            chrom, start, end = re.split(":|-", gap)
            #chrom = chrom[:-len("_Tb427v10")]
            gap_name = chrom + ":" + str(int(start)//1000) + "kbp"
            gap_names.append(gap_name)
        gap_spanning_reads[readname] = " ".join(gap_names)

## post process the data

In [110]:

ref_dict = {x: y[0] for x, y in zip(ref_names, ref_dev)}
fixed_dict = {x: y[0] for x, y in zip(fixed_names, fixed_dev)}

# filter reads
# remove those where the disrance has not changed
filtered = set()
for read_name, distance in zip(fixed_names, fixed_dev):
    if not read_name in gap_spanning_reads:
       filtered.add(read_name)
    # if read_name in ref_dict and ref_dict[read_name] == distance:
    #     filtered.add(read_name)
    pass
    
readnames = [n for n in fixed_names if n in ref_dict and not n in filtered]



# figure out gap groups
gap_groups = set()
for gap in gap_spanning_reads.values():
    gap_groups.add(gap)
gap_colors = {}
for idx, gap in enumerate(gap_groups):
    if len(gap.split()) > 1:
        gap_colors[gap] = "black"
    else:
        gap_colors[gap] = COLOR_PALETTE[idx % len(COLOR_PALETTE)]
read_colors = {}
for read in readnames:
    read_colors[read] = gap_colors[gap_spanning_reads[read]] if read in gap_spanning_reads else "black"


## Plot the results

In [111]:
output_file("../data/out/virtual_paired_read_dist/compare.html")
f = figure(title="Comparison of distance deviations", x_axis_label="Deviation on reference genome", y_axis_label="Deviation on improved genome", 
           tooltips=[("", "@h"), ("", "@r")])

f.line(x=[-1.5*10**5, 0.5*10**5], y=[-1.5*10**5, 0.5*10**5], color="black")

f.scatter(x="x", y="y", line_color=None, fill_color="c", size=9, alpha=0.4, 
          source=ColumnDataSource(data={
                "x": [ref_dict[n] for n in readnames], 
                "y": [fixed_dict[n] for n in readnames],
                "c": [read_colors[n] for n in readnames],
                "h": [gap_spanning_reads[n] if n in gap_spanning_reads else "none" for n in readnames],
                "r": [n for n in readnames]
            }
                ))

show(f, notebook_handle=True)
save(f)

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1121/vscode-ipc-4acb4920-1b03-45a5-8cdf-593ffa8f791d.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1121/vscode-ipc-4acb4920-1b03-45a5-8cdf-593ffa8f791d.sock'
}


'/ladsie/project/ladsie_019/claudia/ont_assembly_improvement/data/out/virtual_paired_read_dist/compare.html'

## check some individual gaps

In [112]:
#GAP_TO_CHECK = "Chr10_3A:1256kbp"
# GAP_TO_CHECK = "Chr11_core:263kbp"
#GAP_TO_CHECK = "Chr7_core:1789kbp" # on the cores there are two populations
#GAP_TO_CHECK = "Chr5_3B:286kbp"
# GAP_TO_CHECK = "Chr11_3B:266kbp"
#GAP_TO_CHECK = "Chr6_A:2589kbp"
GAP_TO_CHECK = "Chr5_A:1507kbp"
# GAP_TO_CHECK = "BES17:65kbp"
# GAP_TO_CHECK = "Chr11_3A:497kbp"

f = figure(title="Comparison of distance deviations", x_axis_label="Reference", y_axis_label="Fixed", 
           tooltips=[("", "@h"), ("", "@r")])

picked_readnames = [n for n in readnames if n in gap_spanning_reads and gap_spanning_reads[n] == GAP_TO_CHECK]

f.line(x=[-1.5*10**5, 0.5*10**5], y=[-1.5*10**5, 0.5*10**5], color="black")
f.scatter(x="x", y="y", line_color=None, fill_color="c", size=9, alpha=0.4, 
          source=ColumnDataSource(data={
                "x": [ref_dict[n] for n in picked_readnames], 
                "y": [fixed_dict[n] for n in picked_readnames],
                "c": [read_colors[n] for n in picked_readnames],
                "h": [gap_spanning_reads[n] for n in picked_readnames],
                "r": picked_readnames
            }
                ))

show(f, notebook_handle=True)

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1121/vscode-ipc-4acb4920-1b03-45a5-8cdf-593ffa8f791d.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1121/vscode-ipc-4acb4920-1b03-45a5-8cdf-593ffa8f791d.sock'
}


## get read clusters -> turn into table of "correctly" expanded gaps

In [113]:
#GAP_TO_CHECK = "Chr10_3A:1256kbp"
#GAP_TO_CHECK = "Chr11_core:263kbp"
#GAP_TO_CHECK = "Chr11_3B:266kbp"
GAP_TO_CHECK = "Chr11_A:4918kbp" # on the cores there are two populations
#GAP_TO_CHECK = "Chr5_3B:286kbp"

f = figure(title="looking at how to best cluster", x_axis_label="Ref fixed difference", y_axis_label="ref fixed sum", 
           tooltips=[("", "@h"), ("", "@r")])

picked_readnames = [n for n in readnames if n in gap_spanning_reads and gap_spanning_reads[n] == GAP_TO_CHECK]

f.scatter(x="x", y="y", line_color=None, fill_color="c", size=9, alpha=0.4, 
          source=ColumnDataSource(data={
                "x": [ref_dict[n] - fixed_dict[n] for n in picked_readnames], 
                "y": [fixed_dict[n] + ref_dict[n] for n in picked_readnames],
                "c": [read_colors[n] for n in picked_readnames],
                "h": [gap_spanning_reads[n] for n in picked_readnames],
                "r": picked_readnames
            }
                ))

show(f, notebook_handle=True)

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1121/vscode-ipc-4acb4920-1b03-45a5-8cdf-593ffa8f791d.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1121/vscode-ipc-4acb4920-1b03-45a5-8cdf-593ffa8f791d.sock'
}


In [114]:

def extract_reads_for_gap(gap_name):
    return [n for n in readnames if n in gap_spanning_reads and gap_spanning_reads[n] == gap_name]

def cluster(l, key, max_dif):
    l.sort(key=key)

    clustered = [[l[0]]]
    if len(l) > 1:
        for read in l[1:]:
            if abs(key(read) - key(clustered[-1][-1])) > max_dif:
                clustered.append([])
            clustered[-1].append(read)
    return clustered

def cluster_reads(readnames, max_ref_fixed_diff = 10, max_ref_fixed_sum = 1000):
    return [c for l in cluster(readnames, lambda x: ref_dict[x] - fixed_dict[x], max_ref_fixed_diff) 
               for c in cluster(l, lambda x: ref_dict[x] + fixed_dict[x], max_ref_fixed_sum)]

def filter_clusters(clusters):
    return [c for c in clusters if len(c) > 3]

def get_mean_deviation_in_clusters(clusters, in_fixed=True):
    return [sum([fixed_dict[n] if in_fixed else ref_dict[n] for n in cluster]) / len(cluster) for cluster in clusters]

In [115]:
#GAP_TO_CHECK = "Chr10_3A:1256kbp"
#GAP_TO_CHECK = "Chr11_core:263kbp"
GAP_TO_CHECK = "Chr11_A:4918kbp" # on the cores there are two populations
#GAP_TO_CHECK = "Chr5_3B:286kbp"

picked_readnames = extract_reads_for_gap(GAP_TO_CHECK)


def cluster(l, key, max_dif):
    l.sort(key=key)

    clustered = [[l[0]]]
    if len(l) > 1:
        for read in l[1:]:
            if abs(key(read) - key(clustered[-1][-1])) > max_dif:
                clustered.append([])
            clustered[-1].append(read)
    return clustered

clustered = cluster_reads(picked_readnames)


print(clustered)
print([len(c) for c in clustered])

clustered = filter_clusters(clustered)

print(get_mean_deviation_in_clusters(clustered))
print(get_mean_deviation_in_clusters(clustered, False))


[['e61d63ea-2b70-487d-a0eb-bba4b549341f', '137129fd-2e7f-4efb-a1fd-0c7d4ea5edab', '9bed4145-a3fd-4672-8991-03406cda49d4', '7ecfa24f-4b2a-4762-92cd-fee5a4abdb55', '2de4a0e6-4f79-4bba-a62a-c90bb365a167', '21009d26-1b33-4ba9-b718-6e8c86057c72', 'fd16ca02-0281-4bd5-8b6b-7e0323998994', '229dad4d-3dbf-4c9f-9e9f-25d619130eef', '2d986e3c-b156-4889-bdac-a49f50d98905', 'a9f25b7f-4909-4335-a8c9-ff5dec459de1', 'd5741f3e-c34c-46cb-bf62-18894f153ea7', '17af5f35-2ee4-40ab-92ac-854fab35c7f0', '90b8f85f-c1bb-4053-9e1d-85b5fe22d015', '1d572ad9-acfc-4a60-abf0-d2da0a545aa5', '5eadf8a8-5db7-4e44-87fd-2267b7d32782', 'e36b7e9e-26d3-421b-b336-d7223e4abf79', '315d2b25-25c2-4603-a5b6-1f2d2822563a', '9805c5af-446d-4a1c-8aa6-2a1365f0003b', 'bf469545-e0ec-4331-89a3-30e7a77b09f4', '9199eeaa-d2c8-449e-914a-ac7e1bb27bb6', '00c8f215-ce90-44b3-b5e4-829feb6393d0', 'ec22ecb8-e4fb-4454-8561-12599e4ecd32'], ['b49c29f2-d6f3-4ff3-9a99-17d2defcff85'], ['b3f9f1c3-b25c-490c-8bf7-a3fcdb29b83d']]
[22, 1, 1]
[10686.045454545454]
[

In [116]:
gap_closed_if_fixed_dev_smaller_than = 5000


gap_names = gap_pos.keys()

closed_gaps = 0
print("correct", "#supp", "#contra", "dev", "fxd_dev", "other", "has_cluster", "name", sep="\t")
with open("../data/in/analysis_in/closed_gaps_analysis.gff", "w") as file_out:
    for gap in sorted(gap_names):
        has_cluster = not gap in gap_without_cluster
        read_names = extract_reads_for_gap(gap)
        if len(read_names) > 0:
            read_clusters = filter_clusters(cluster_reads(read_names))
            cluster_fixed = get_mean_deviation_in_clusters(read_clusters)
            gap_sizes = get_mean_deviation_in_clusters(read_clusters, False)
            gap_closed = False
            gap_idx = 0
            min_fixed = float("inf")
            for idx, x in enumerate(cluster_fixed):
                if abs(x) < gap_closed_if_fixed_dev_smaller_than and abs(x) < min_fixed:
                    gap_closed = True
                    gap_idx = idx
                    min_fixed = abs(x)
            if gap_closed:
                print("Yes", len(read_clusters[idx]), len(read_names) - len(read_clusters[idx]), int(gap_sizes[idx]), 
                    int(cluster_fixed[idx]), len(gap_sizes) > 1, has_cluster, gap, sep="\t")
                closed_gaps += 1
                chrom, start, end = gap_pos[gap]
                file_out.write("\t".join([chrom + "_Tb427v10", ".", "fixedgap", str(start), str(end), ".", ".", ".", 
                                          "estimated_length=1000;gap_type=within scaffold;closed_correctly=true"]) + "\n" )
            else:
                print("No" if len(read_names) > 5 else "?", "", len(read_names), "", "", "", has_cluster, gap, sep="\t")
                file_out.write("\t".join([chrom + "_Tb427v10", ".", "notenoughdatagap", str(start), str(end), ".", ".", ".", 
                                          "estimated_length=1000;gap_type=within scaffold;not_enough_data=true"]) + "\n" )
        else:
            print("?", "", "", "", "", "", has_cluster, gap, sep="\t")
            file_out.write("\t".join([chrom + "_Tb427v10", ".", "notenoughdatagap", str(start), str(end), ".", ".", ".", 
                                        "estimated_length=1000;gap_type=within scaffold;not_enough_data=true"]) + "\n" )
print()
print()
print("closed", closed_gaps, "out of", len(gap_names), "gaps")

correct	#supp	#contra	dev	fxd_dev	other	has_cluster	name
Yes	14	2	-1091	-3062	False	True	BES17:65kbp
?		3				False	BES2:62kbp
Yes	21	0	1125	128	False	False	Chr10_A:4101kbp
Yes	14	1	-6040	134	False	True	Chr10_A:76kbp
Yes	11	4	1043	45	False	False	Chr10_B:4083kbp
Yes	10	2	-44272	-2734	False	True	Chr10_B:5341kbp
Yes	19	1	-11365	182	False	True	Chr10_B:58kbp
Yes	6	1	-2951	-2951	False	True	Chr11_A:14kbp
?						False	Chr11_A:279kbp
Yes	27	1	-104	81	False	True	Chr11_A:4651kbp
No		24				False	Chr11_A:4918kbp
?		5				True	Chr11_A:4971kbp
?						False	Chr11_B:296kbp
Yes	11	0	1065	69	False	False	Chr11_B:32kbp
Yes	17	3	-10477	83	False	True	Chr11_B:4669kbp
Yes	18	0	-23788	336	False	True	Chr11_B:4955kbp
?		2				False	Chr11_B:5168kbp
?						False	Chr11_B:5455kbp
?		1				False	Chr1_A:100kbp
?						False	Chr1_A:1149kbp
?		1				False	Chr1_A:1212kbp
?		1				False	Chr1_A:2357kbp
Yes	35	2	-3970	113	False	True	Chr1_A:3037kbp
?						False	Chr1_A:711kbp
Yes	9	0	1099	101	False	True	Chr1_A:946kbp
Yes	26	0	-1686	1